# Установка gensim на воркер

Установка conda:
* `https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh`
* `chmod +x Anaconda3-2019.10-Linux-x86_64.sh`
* `./Anaconda3-2019.10-Linux-x86_64.sh`  (если случайно запустить с sudo, то потом [поменять владельца](https://github.com/conda/conda/issues/7267#issuecomment-498651734))
* отключить авто-init: `conda config --set auto_activate_base false`

Gensim:
* `conda install gensim`

Jupyter:
* `conda install ipykernel`
* запуск сервера: `jupyter notebook --no-browser --port=8888 --ip='::'`
* выбрать кернел



In [1]:
import gensim

# Качаем данные с википедии

In [2]:
# https://dumps.wikimedia.org/ruwiki/20191220/
!wget https://dumps.wikimedia.org/ruwiki/20191220/ruwiki-20191220-pages-articles-multistream.xml.bz2

--2019-12-25 12:51:41--  https://dumps.wikimedia.org/ruwiki/20191220/ruwiki-20191220-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|2620:0:861:1:208:80:154:7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3992310344 (3,7G) [application/octet-stream]
Saving to: `ruwiki-20191220-pages-articles-multistream.xml.bz2'

100%[====================================>] 3 992 310 344 4,88M/s   in 12m 50s 

2019-12-25 13:04:33 (4,94 MB/s) - `ruwiki-20191220-pages-articles-multistream.xml.bz2' saved [3992310344/3992310344]



In [1]:
from datetime import datetime

log_filename = 'DL_chapter_7_word2vec_in_memory.log'

def log(msg):
    msg = '[{}] {}'.format(datetime.now(), msg)
    print(msg)
    with open(log_filename, 'a') as fout:
        fout.write(msg + '\n')

# Загрузка и предобработка данных

*Токенизация*: разбиение потока символов на слова (+ очистка от лишних символов)

*Лемматизация*: приведение слов к начальным формам. Мы не будем делать, т.к. во-первых хотим посмотреть на всякие эффекты, а во-вторых, у нас и так достаточно данных

In [2]:
from gensim.corpora.wikicorpus import WikiCorpus

wiki = WikiCorpus('ruwiki-20191220-pages-articles-multistream.xml.bz2', dictionary=False)

Данные не читаются с диска в память, а благодаря dictionary=False не строится и словарь

*Биграмма* — это пара слов `(w1,w2)`, которые вместе встречаются аномально часто, то есть совместная вероятность их появления `p(w1,w2)`
в тексте существенно выше, чем `p(w1)p(w2)`

Есть разные алгоритмы выделения биграмм, но в Gensim уже есть реализация

In [3]:
def get_texts_step(step=100):
    # get every [step]th text
    for i, text in enumerate(wiki.get_texts()):
        if i % step == 0:
            yield text

log("Started loading texts")
all_texts = list(wiki.get_texts())
log("Started loading texts")


def get_texts():
    # get texts from memory
    for text in all_texts:
        yield text

[2019-12-25 16:12:57.572576] Started loading texts
[2019-12-25 16:41:07.762148] Started loading texts


In [4]:
from gensim.models.phrases import Phrases, Phraser

# генератор биграм

log("Started building bigrams")
bigram = Phrases(get_texts())
bigram_transformer = Phraser(bigram)
log("Finished building bigrams")


def text_generator_bigram():
    for text in get_texts():
        yield bigram_transformer[ [ word for word in text ] ]   # .decode('utf-8')

[2019-12-25 16:57:19.300192] Started building bigrams
[2019-12-25 17:31:09.662020] Finished building bigrams


In [5]:
# генератор триграм

log("Started building trigrams")
trigram = Phrases(text_generator_bigram())
trigram_transformer = Phraser(trigram)
log("Finished building trigrams")

def text_generator_trigram():
    for text in get_texts():
        yield trigram_transformer[ bigram_transformer[ [ word for word in text ] ] ]   # .decode('utf-8')

[2019-12-25 17:31:09.669413] Started building trigrams
[2019-12-25 18:29:54.008252] Finished building trigrams


# Обучаем word2vec на последовательности триграмм

## Обучаю модель size=100, epochs=25

In [6]:
log("Started counting examples")
total_examples = 0
for trigram in text_generator_trigram():
    total_examples += 1
log("Finished counting examples, result: {}".format(total_examples))

[2019-12-25 18:29:54.014886] Started counting examples
[2019-12-25 19:26:06.959298] Finished counting examples, result: 1331252


In [7]:
from gensim.models.word2vec import Word2Vec

model_100 = Word2Vec(size=100, window=7, min_count=10, workers=32)

log("Started building vocab")
model_100.build_vocab( text_generator_trigram() )
log("Finished building vocab")

[2019-12-25 19:26:06.966888] Started building vocab
[2019-12-25 20:37:31.822850] Finished building vocab


In [8]:
log("Started training")
model_100.train( text_generator_trigram(), total_examples=total_examples, epochs=25 )
log("Finished training")

[2019-12-25 20:37:31.828769] Started training
[2019-12-25 21:39:03.319412] Finished training


In [9]:
model_100.save('DL_chapter_7_word2vec_model_size_100_percent_100_epochs_25')

* size -- размерность вектора представления
* window -- размер окна локального контекста
* min_count -- минимальная частота встречаемости, чтобы мы рассматривали слово
* workers -- параллелизация

## Обучаю модель size=500, epochs=25

In [10]:
model_500 = Word2Vec(size=500, window=7, min_count=10, workers=32)

log("Started building vocab")
model_500.build_vocab( text_generator_trigram() )
log("Finished building vocab")

[2019-12-25 21:39:12.828759] Started building vocab
[2019-12-25 22:48:20.032481] Finished building vocab


In [11]:
log("Started training")
model_500.train( text_generator_trigram(), total_examples=total_examples, epochs=25 )
log("Finished training")

[2019-12-25 22:48:20.039712] Started training
[2019-12-25 23:50:28.923686] Finished training


In [12]:
model_500.save('DL_chapter_7_word2vec_model_size_500_percent_100_epochs_25')

## Обучаю модель size=500, epochs=200

In [13]:
model_500_200 = Word2Vec(size=500, window=7, min_count=10, workers=32)

log("Started building vocab")
model_500_200.build_vocab( text_generator_trigram() )
log("Finished building vocab")

[2019-12-25 23:50:49.341133] Started building vocab
[2019-12-26 00:59:23.792330] Finished building vocab


In [14]:
log("Started training")
model_500_200.train( text_generator_trigram(), total_examples=total_examples, epochs=200 )
log("Finished training")

[2019-12-26 00:59:23.830482] Started training
[2019-12-26 02:01:54.071193] Finished training


In [15]:
model_500_200.save('DL_chapter_7_word2vec_model_size_500_percent_100_epochs_200')

In [16]:
model_500_200.wv.most_similar(
  ['актёр', 'женщина'],
  ['мужчина']
)

[('актриса', 0.6711472868919373),
 ('ведущая_актриса', 0.5662055611610413),
 ('актер', 0.5578432679176331),
 ('сценарист_режиссёр', 0.5533250570297241),
 ('киноактриса', 0.5437122583389282),
 ('благодаря_ролям', 0.5435642600059509),
 ('снимавшаяся', 0.5400126576423645),
 ('балерина', 0.5322019457817078),
 ('театра_кино', 0.527100682258606),
 ('танцовщица', 0.5250095725059509)]

In [17]:
model_500_200.wv.most_similar(
  ['математик', 'женщина'],
  ['мужчина']
)

[('доктор_философии', 0.6313427686691284),
 ('итальянский_физик', 0.6195719242095947),
 ('учёная', 0.6180987358093262),
 ('немецкий_философ', 0.6100102663040161),
 ('немецкий_математик', 0.6081817746162415),
 ('филолог', 0.6068433523178101),
 ('физиолог', 0.6051205396652222),
 ('лингвист', 0.6050077676773071),
 ('доктор_филологических_наук', 0.6020523309707642),
 ('американский_математик', 0.5991174578666687)]